In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import timedelta
from functools import reduce

In [2]:

# Define the ticker symbols
tickers = {
    'BTC-USD': 'BTC',
    '^TNX': 'RF',
    'SPY': 'SPY',
    '^VIX': 'VIX'
}

# Define the date range
lookback_period = 150
start_date = pd.to_datetime('2019-11-01')
end_date = pd.to_datetime('2024-10-31')

# Initialize an empty dictionary to store DataFrames
data_frames = {}

# Loop through each ticker and download the data
for ticker, name in tickers.items():
    # Download the data
    data = yf.download(ticker, start=start_date-timedelta(days=lookback_period*2), end=end_date)
    
    # Reset index to make 'Date' a column
    data = data.reset_index()
    
    # Convert 'Date' to datetime.date format
    data['date'] = pd.to_datetime(data['Date']).dt.date
    
    # Select relevant columns and rename them
    if ticker == 'BTC-USD':
        # For Bitcoin, keep 'Close' and 'Volume'
        data = data[['date', 'Close', 'Volume']]
        data.rename(columns={'Close': 'PRC', 'Volume': 'Vol'}, inplace=True)
    elif ticker == '^TNX':
        # For 10-year Treasury Yield, keep 'Close' and adjust yield to percentage
        data = data[['date', 'Close']]
        data.rename(columns={'Close': 'RF'}, inplace=True)
        # Convert yield from percentage points to decimal (e.g., 15.00 to 0.15)
        data['RF'] = data['RF'] / 100
    else:
        # For SPY and VIX, keep 'Adj Close'
        data = data[['date', 'Adj Close']]
        data.rename(columns={'Adj Close': name}, inplace=True)
    
    # Store the DataFrame in the dictionary
    data_frames[name] = data

# Merge all DataFrames on 'date'

dfs = list(data_frames.values())
merged_data = reduce(lambda left, right: pd.merge(left, right, on='date', how='inner'), dfs)
merged_data.columns = merged_data.columns.get_level_values(0) 

merged_data


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_34368/2167053777.py:50: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  merged_data = reduce(lambda left, right: pd.merge(left, right, on='date', how='inner'), dfs)
/tmp/ipykernel_34368/2167053777.py:50: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  merged_data = reduce(lambda left, right: pd.merge(left, right, on='date', how='inner'), dfs)
/tmp/ipykernel_34368/2167053777.py:50: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  merged_data = reduce(lambda left, right: pd.merge(left, right, on='date', how='inner'

Price,date,PRC,Vol,RF,SPY,VIX
0,2019-01-07,4025.248291,5228625637,0.02682,232.238022,21.400000
1,2019-01-08,4030.847900,5306593305,0.02716,234.419968,20.469999
2,2019-01-09,4035.296387,5115905225,0.02728,235.515579,19.980000
3,2019-01-10,3678.924561,6874143796,0.02731,236.346329,19.500000
4,2019-01-11,3687.365479,5538712865,0.02701,236.437653,18.190001
...,...,...,...,...,...,...
1460,2024-10-24,68161.054688,31414428647,0.04200,579.239990,19.080000
1461,2024-10-25,66642.414062,41469984306,0.04232,579.039978,20.330000
1462,2024-10-28,69907.757812,38799856657,0.04278,580.830017,19.799999
1463,2024-10-29,72720.492188,58541874402,0.04274,581.770020,19.340000


In [3]:
# create the signal for the trading range breakout
merged_data['SUP'] = merged_data['PRC'].shift(1).rolling(window=lookback_period).min()
merged_data['RES'] = merged_data['PRC'].shift(1).rolling(window=lookback_period).max()
merged_data

Price,date,PRC,Vol,RF,SPY,VIX,SUP,RES
0,2019-01-07,4025.248291,5228625637,0.02682,232.238022,21.400000,NaN,NaN
1,2019-01-08,4030.847900,5306593305,0.02716,234.419968,20.469999,NaN,NaN
2,2019-01-09,4035.296387,5115905225,0.02728,235.515579,19.980000,NaN,NaN
3,2019-01-10,3678.924561,6874143796,0.02731,236.346329,19.500000,NaN,NaN
4,2019-01-11,3687.365479,5538712865,0.02701,236.437653,18.190001,NaN,NaN
...,...,...,...,...,...,...,...,...
1460,2024-10-24,68161.054688,31414428647,0.04200,579.239990,19.080000,53948.753906,71631.359375
1461,2024-10-25,66642.414062,41469984306,0.04232,579.039978,20.330000,53948.753906,71631.359375
1462,2024-10-28,69907.757812,38799856657,0.04278,580.830017,19.799999,53948.753906,71631.359375
1463,2024-10-29,72720.492188,58541874402,0.04274,581.770020,19.340000,53948.753906,71631.359375


In [4]:
merged_data = merged_data.dropna()
merged_data = merged_data[(merged_data['date'] >= pd.to_datetime(start_date).date()) & (merged_data['date'] <= pd.to_datetime(end_date).date())]
merged_data = merged_data.reset_index(drop=True)

merged_data

Price,date,PRC,Vol,RF,SPY,VIX,SUP,RES
0,2019-11-01,9261.104492,24324691031,0.01728,283.391846,12.300000,4158.183105,13016.231445
1,2019-11-04,9412.612305,26170255634,0.01786,284.530396,12.830000,4879.877930,13016.231445
2,2019-11-05,9342.527344,26198609048,0.01866,284.215729,13.100000,4922.798828,13016.231445
3,2019-11-06,9360.879883,23133895765,0.01814,284.280548,12.620000,4922.798828,13016.231445
4,2019-11-07,9267.561523,22700383839,0.01926,285.280243,12.730000,5036.681152,13016.231445
...,...,...,...,...,...,...,...,...
1252,2024-10-24,68161.054688,31414428647,0.04200,579.239990,19.080000,53948.753906,71631.359375
1253,2024-10-25,66642.414062,41469984306,0.04232,579.039978,20.330000,53948.753906,71631.359375
1254,2024-10-28,69907.757812,38799856657,0.04278,580.830017,19.799999,53948.753906,71631.359375
1255,2024-10-29,72720.492188,58541874402,0.04274,581.770020,19.340000,53948.753906,71631.359375


In [5]:
merged_data['signal'] = np.where(merged_data['PRC'] > merged_data['RES'], 1, 0)  # Buy signal
merged_data['signal'] = np.where(merged_data['PRC'] < merged_data['SUP'], -1, merged_data['signal'])  # Sell signal
merged_data

Price,date,PRC,Vol,RF,SPY,VIX,SUP,RES,signal
0,2019-11-01,9261.104492,24324691031,0.01728,283.391846,12.300000,4158.183105,13016.231445,0
1,2019-11-04,9412.612305,26170255634,0.01786,284.530396,12.830000,4879.877930,13016.231445,0
2,2019-11-05,9342.527344,26198609048,0.01866,284.215729,13.100000,4922.798828,13016.231445,0
3,2019-11-06,9360.879883,23133895765,0.01814,284.280548,12.620000,4922.798828,13016.231445,0
4,2019-11-07,9267.561523,22700383839,0.01926,285.280243,12.730000,5036.681152,13016.231445,0
...,...,...,...,...,...,...,...,...,...
1252,2024-10-24,68161.054688,31414428647,0.04200,579.239990,19.080000,53948.753906,71631.359375,0
1253,2024-10-25,66642.414062,41469984306,0.04232,579.039978,20.330000,53948.753906,71631.359375,0
1254,2024-10-28,69907.757812,38799856657,0.04278,580.830017,19.799999,53948.753906,71631.359375,0
1255,2024-10-29,72720.492188,58541874402,0.04274,581.770020,19.340000,53948.753906,71631.359375,1


In [6]:
# testing

data = merged_data

# Initialize simulation variables
cb = 0.002  # Buy commission
cs = 0.002  # Sell commission
T = len(data)  # Total number of time steps
nUSD = 1.0  # Initial USD balance
nBTC = 0.0  # Initial BTC balance
CR = [1.0]  # Cumulative return (start with 1)

# Trading simulation loop
for t in range(T - 1):
    # Get the signal for the current time step
    signal = data.loc[t, 'signal']
    price_t = data.loc[t, 'PRC']
    price_t1 = data.loc[t + 1, 'PRC']
    
    if signal == 1 and nBTC == 0:  # Buy condition
        nBTC = nUSD * (1 - cb) / price_t
        nUSD = 0
    elif signal == -1 and nBTC > 0:  # Sell condition
        nUSD = nBTC * (1 - cs) * price_t
        nBTC = 0
    # elif signal == -1 and nBTC == 0:  # Short condition
     #   nBTCb = nUSD / price_t
      #  nUSD = nUSD + nBTCb * (1 - cs) * price_t - nBTCb * price_t1 / (1 - cb)
    
    # Update cumulative return
    CR.append(nUSD + nBTC * price_t1 * (1 - cs))

# Save cumulative return to the data frame
data['cumulative_return'] = CR

In [7]:
data

Price,date,PRC,Vol,RF,SPY,VIX,SUP,RES,signal,cumulative_return
0,2019-11-01,9261.104492,24324691031,0.01728,283.391846,12.300000,4158.183105,13016.231445,0,1.000000
1,2019-11-04,9412.612305,26170255634,0.01786,284.530396,12.830000,4879.877930,13016.231445,0,1.000000
2,2019-11-05,9342.527344,26198609048,0.01866,284.215729,13.100000,4922.798828,13016.231445,0,1.000000
3,2019-11-06,9360.879883,23133895765,0.01814,284.280548,12.620000,4922.798828,13016.231445,0,1.000000
4,2019-11-07,9267.561523,22700383839,0.01926,285.280243,12.730000,5036.681152,13016.231445,0,1.000000
...,...,...,...,...,...,...,...,...,...,...
1252,2024-10-24,68161.054688,31414428647,0.04200,579.239990,19.080000,53948.753906,71631.359375,0,9.025988
1253,2024-10-25,66642.414062,41469984306,0.04232,579.039978,20.330000,53948.753906,71631.359375,0,8.824888
1254,2024-10-28,69907.757812,38799856657,0.04278,580.830017,19.799999,53948.753906,71631.359375,0,9.257289
1255,2024-10-29,72720.492188,58541874402,0.04274,581.770020,19.340000,53948.753906,71631.359375,1,9.629756


In [8]:
(data['SPY'].iloc[-1] / data['SPY'].iloc[0]) - 1

np.float64(1.0466714852947132)

In [9]:
# Calculate excess returns
data['excess_return'] = data['cumulative_return'] - data['RF'] / 252

# Calculate mean and standard deviation of excess returns
mean_excess_return = data['excess_return'].mean()
std_excess_return = data['excess_return'].std()

# Calculate Sharpe Ratio
sharpe_ratio = mean_excess_return / std_excess_return